## Word Normalization

### José Pablo Kiesling Lange

In [1]:
import re
from collections import Counter

In [2]:
corpus_path = "data/escher_comments.txt"

In [3]:
with open(corpus_path, "r", encoding="utf-8") as file:
    corpus = file.readlines()

In [4]:
corpus = [line.strip() for line in corpus if line.strip()]

### Estandarización

Para ver la efectivdad de la estandarización, se mostrará las 10 palabras más frecuentes del corpus antes y después de la estandarización. El objetivo es ver si hay modificación en la cantidad de dichas palabras o si una nueva palabra aparece con frecuencia.

In [5]:
def get_most_common_words(corpus, n=10):
    words = [word for line in corpus for word in line.split()]
    most_common = Counter(words).most_common(n)
    return most_common

In [6]:
most_common_words = get_most_common_words(corpus, n=10)
most_common_words

[('que', 45),
 ('de', 39),
 ('la', 32),
 ('un', 27),
 ('una', 26),
 ('el', 17),
 ('en', 16),
 ('esfera', 15),
 ('reflejo', 15),
 ('y', 14)]

#### Case Folding

Se hace una función que dado un corpus revisa si posee la misma palabra en mayúscula y minúscula.

In [7]:
def get_upper_lower_words(corpus):
    words = set(word for line in corpus for word in line.split())
    upper_lower_words = set()
    for word in words:
        if word.lower() in words and word.upper() in words:
            upper_lower_words.add(word)
    return upper_lower_words

In [8]:
upper_lower_words = get_upper_lower_words(corpus)
upper_lower_words

{'A', 'a'}

Dado que sí hay por lo menos una palabra que aparece en ambas formas, se procede a hacer un case folding (pasar todas las palabras a minúscula).

In [9]:
def case_folding(corpus):
    return [line.lower() for line in corpus]

#### Remove Punctuation

Dado que en las instrucciones para generar el corpus se pidió mínimo 2 oraciones, se asume que mínimo una de las oraciones tiene puntuación. Por lo tanto, se procede a eliminar la puntuación del corpus.

Esto se hace para evitar que palabras como `la,` o `la.` se cuenten como palabras diferentes a `la`.

In [12]:
def remove_punctuation(corpus):
    return [re.sub(r'[^\w\s]', '', line) for line in corpus]

In [13]:
def standarizate(corpus):
    corpus = case_folding(corpus)
    corpus = remove_punctuation(corpus)
    return corpus

In [14]:
corpus = standarizate(corpus)

### Corpus ya estandarizado

In [15]:
most_common_words = get_most_common_words(corpus, n=10)
most_common_words

[('que', 45),
 ('de', 39),
 ('la', 34),
 ('un', 30),
 ('una', 28),
 ('esfera', 18),
 ('en', 18),
 ('el', 17),
 ('reflejo', 16),
 ('se', 15)]

Como se puede apreciar:
- la palabra `la` se incrementó en `2`ocurrencias
- la palabra `un` se incrementó en `3` ocurrencias
- la palabra `una` se incrementó en `3`ocurrencias
- la palabra `esfera` se incrementó en `3` ocurrencias y subió al `6to` lugar
- la palabra `en` se incrementó en `2` ocurrencias y subió al `7mo` lugar
- la palabra `reflejo` se incrementó en `1` ocurrencia
- la palabra `se` **ingresó** al top 10

Por lo tanto, la estandarización ha tenido un efecto positivo en la, ya que ha incrementado la cantidad de ocurrencias de palabras comunes y ha eliminado las variaciones causadas por puntuación.

Algo a recalcar es la ausencia de la estandarización de palabras con o sin tilde. Esto es porque se entrenará el modelo `BPE` en español.